# **DATA CLEANING NOTEBOOK**

## Objectives

* Clean input data
  * Add 3 variables: 'Damage', 'Amount', 'Day of week'
  * Replace missing variables, and group: 'Vict Sex', 'Vict Descent', 'Premis Cd' and 'Premis Desc', 'Weapon used Cd' and 'Weapon Desc', 'Cross Street'
  * Dropping variables: DR_NO/Date Rptd/Rpt Dist No/Part 1-2/Mocodes/Status/Status Desc/Crm Cd 1/Crm Cd 2/Crm Cd 3/Crm Cd 4
* Save data

## Inputs

* The input data is a csv file called Crime_Data_from_2020_to_Present.csv
* We also use a csv file, created just for this case, called "Crm Cd Desc analyses.csv"

## Outputs

* The output data is a csv file called dataPP5.csv 

## Additional Comments

* ...


---

# Change working directory

* access the current directory
* make the parent of the current directory the new current directory

In [1]:
import os
current_dir = os.getcwd()
os.chdir(os.path.dirname(current_dir))
current_dir = os.getcwd()
current_dir

'/workspace/GazetteAnaliticsTools'

---

# Load Data

In [2]:
import pandas as pd
df = pd.read_csv("inputs/contentdata/gazette_contentdata_jupyter.csv")
df.head(3)


,id,entryType,language,publicationDate,legalRemedy,cantons,title_en,journal_date,publication_text,company_name,...,company_code13,company_seat,company_legalForm,company_street_and_number,company_zip_and_town,company_purpose,company_capital_nominal,company_capital_paid,company_optingout,company_deletiondate
0,84a362d7-97db-4298-9cb5-fd729a14ff9f,Change,fr,2025-05-22,La mutation de l'entité juridique mentionnée a...,GE,"Change TRIVAL-SERVICES SA, Genève",2025-05-19,"TRIVAL-SERVICES SA, à Genève, CHE-198.815.670 ...",TRIVAL-SERVICES SA,...,CH66019500175,Genève,106,Rue Adrien-Lachenal 20,1207 Genève,en Suisse et à l'étranger: toutes activités de...,102000.00,102000.00,false,no data
1,ddd70bcd-e502-444c-8243-f21dcf98a4d4,Change,fr,2025-05-22,La mutation de l'entité juridique mentionnée a...,GE,"Change LENNYS SA, Genève",2025-05-19,"LENNYS SA, à Genève, CHE-244.696.821 (FOSC du ...",LENNYS SA,...,CH66046860240,Genève,106,Rue du Mont-Blanc 7,1201 Genève,la prise de participations directes et indirec...,100000.00,100000.00,false,no data
2,d13a8152-4564-4c72-bd91-cff657ce9320,Change,fr,2025-05-22,La mutation de l'entité juridique mentionnée a...,GE,"Change TEMENOS AG, Lancy",2025-05-19,"TEMENOS AG, à Lancy, CHE-109.066.419 (FOSC du ...",TEMENOS AG,...,CH16030039537,Lancy,106,Esplanade de Pont-Rouge 9C,1212 Grand-Lancy,"directement ou indirectement l'acquisition, la...",359535735.00,359535735.00,false,no data


---

Cleaning 'date' type columns, in existing CSV (only once before uploading to heroku postgresql 'gazette_contentdata' table)

In [3]:
import pandas as pd

input_path = "inputs/contentdata/gazette_contentdata_jupyter.csv"

# 1. Read the CSV
df = pd.read_csv(input_path)

# 2. List the columns that are DATE in Postgres
date_cols = ["publicationDate", "journal_date", "company_deletiondate"]

# 3. Replace the string "no data" with actual pandas NA
df[date_cols] = df[date_cols].replace("no data", pd.NaT)

# 4. (Optional) ensure they’re proper datetimes; any non-parseable values become NaT
for c in date_cols:
    df[c] = pd.to_datetime(df[c], errors="coerce")

# 5. Overwrite the CSV
df.to_csv(input_path, index=False)

print(f"Cleaned {input_path}: replaced 'no data' with blanks in {date_cols}")

Cleaned inputs/contentdata/gazette_contentdata_jupyter.csv: replaced 'no data' with blanks in ['publicationDate', 'journal_date', 'company_deletiondate']


Cleaning 'numeric' type columns, in existing CSV (only once before uploading to heroku postgresql 'gazette_contentdata' table)

In [4]:
import pandas as pd

# 1. Load the CSV
path = "inputs/contentdata/gazette_contentdata_jupyter.csv"
df = pd.read_csv(path)

# 2. Columns to fix
numeric_cols = ["company_capital_nominal", "company_capital_paid"]

# 3. Replace the literal "no data" with NaN, then coerce to numeric
for col in numeric_cols:
    # turn any “no data” (or other non‐numeric) into NaN
    df[col] = pd.to_numeric(df[col].replace("no data", pd.NA), errors="coerce")

# 4. (Optional) fill NaN with empty so that CSV will produce blank cells
#    — Postgres will interpret blanks as NULL when importing
df[numeric_cols] = df[numeric_cols].fillna("")

# 5. Overwrite the CSV
df.to_csv(path, index=False)

print(f"Cleaned numeric columns {numeric_cols} in {path}")

Cleaned numeric columns ['company_capital_nominal', 'company_capital_paid'] in inputs/contentdata/gazette_contentdata_jupyter.csv


Cleaning 'boolean' type columns, in existing CSV (only once before uploading to heroku postgresql 'gazette_contentdata' table)

In [5]:
import pandas as pd

# 1. Load your CSV
path = "inputs/contentdata/gazette_contentdata_jupyter.csv"
df = pd.read_csv(path, dtype=str)  # read everything as string

# 2. Specify the boolean columns to fix
bool_cols = ["company_optingout"]

for col in bool_cols:
    # 3a. Normalize case and replace "no data" (or any other non-true/false) with pd.NA
    s = df[col].str.lower().replace("no data", pd.NA)
    # 3b. Map "true"->True, "false"->False; anything else becomes pd.NA
    df[col] = s.map({"true": True, "false": False})
    # 4. When writing to CSV, blanks will become empty cells (interpreted as NULL by Postgres)
    df[col] = df[col].where(df[col].notna(), "")

# 5. Overwrite the CSV
df.to_csv(path, index=False)

print(f"Cleaned boolean column(s) {bool_cols} in {path}")

Cleaned boolean column(s) ['company_optingout'] in inputs/contentdata/gazette_contentdata_jupyter.csv
